<a href="https://colab.research.google.com/github/mneedham/data-science-training/blob/master/01_DataLoading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Citation Dataset Loading

In this notebook we're going to load the citation dataset into Neo4j.

First let's import a couple of Python libraries that will help us with this process.

In [ ]:
!pip install py2neo pandas


We'll start by importing py2neo and the pandas library which we'll be using to play around with the data later on.

In [ ]:
from py2neo import Graph
import pandas as pd

Create a [Neo4j 3.4 Sandbox](https://neo4j.com/sandbox-v2/) and paste the credentials into the cell below.


<div align="left">
    <img src="images/sandbox.png" alt="Neo4j 3.4 Sandbox"/>
</div>


In [ ]:
# Change the line of code below to use the IP Address, Bolt Port, and Password of your Sandbox.
# graph = Graph("bolt://<IP Address>:<Bolt Port>", auth=("neo4j", "<Password>")) 

# graph = Graph("bolt://18.234.168.45:33679", auth=("neo4j", "daybreak-cosal-rumbles")) 
graph = Graph("bolt://localhost", auth=("neo4j", "neo")) 

## Create Constraints

First let's create some constraints to make sure we don't import duplicate data:

In [ ]:
display(graph.run("CREATE CONSTRAINT ON (article:Article) ASSERT article.index IS UNIQUE").stats())
display(graph.run("CREATE CONSTRAINT ON (author:Author) ASSERT author.name IS UNIQUE").stats())
display(graph.run("CREATE CONSTRAINT ON (v:Venue) ASSERT v.name IS UNIQUE").stats())

## Loading the data

Now let's load the data into the database. We'll create nodes for Articles, Venues, and Authors.


In [ ]:
query = """
CALL apoc.periodic.iterate(
  'UNWIND ["dblp-ref-0.json", "dblp-ref-1.json", "dblp-ref-2.json", "dblp-ref-3.json"] AS file
   CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/" + file)
   YIELD value WITH value
   WHERE value.venue IN ["Lecture Notes in Computer Science", 
                         "Communications of The ACM",
                         "international conference on software engineering",
                         "advances in computing and communications"]
   return value',
  'MERGE (a:Article {index:value.id})
   SET a += apoc.map.clean(value,["id","authors","references", "venue"],[0])
   WITH a, value.authors as authors, value.references AS citations, value.venue AS venue
   MERGE (v:Venue {name: venue})
   MERGE (a)-[:VENUE]->(v)
   FOREACH(author in authors | 
     MERGE (b:Author{name:author})
     MERGE (a)-[:AUTHOR]->(b))
   FOREACH(citation in citations | 
     MERGE (cited:Article {index:citation})
     MERGE (a)-[:CITED]->(cited))', 
   {batchSize: 1000, iterateList: true});
"""
graph.run(query).to_data_frame()

In the next notebook we'll explore the data that we've imported. 